In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score, recall_score, confusion_matrix

# Bootstrap predictions

In [ ]:
with open('train_y.pkl', 'rb') as f:
    train_y = pickle.load(f)
with open('test_y.pkl', 'rb') as f:
    test_y = pickle.load(f)

In [ ]:
def bootstrap_performance_metrics(prediction_data):
    # Bootstrap the data
    boot_data = resample(prediction_data, stratify=prediction_data['y_true'])

    # Performance metrics
    auc = roc_auc_score(boot_data['y_true'], boot_data['y_prob'])
    sensitivity = recall_score(boot_data['y_true'], boot_data['y_pred'])
    tn, fp, fn, tp = confusion_matrix(boot_data['y_true'], boot_data['y_pred']).ravel()
    specificity = tn / (tn + fp)

    # Collect metrics in dataframe
    bootstrap_df = pd.DataFrame({'AUC': [auc],
                                 'SENSITIVITY': [sensitivity],
                                 'SPECIFICITY': [specificity]})
    return bootstrap_df

def summarize_bootstrap_results(bootstrap_results):    
    alpha = 100-95
    metrics = []
    medians = []
    ci_low = []
    ci_high = []
    
    for col in bootstrap_results.columns:
        metrics.append(col)
        medians.append(np.percentile(bootstrap_results[col], 50))
        ci_low.append(np.percentile(bootstrap_results[col], alpha/2))
        ci_high.append(np.percentile(bootstrap_results[col], 100-alpha/2))

    metrics = pd.DataFrame({'METRIC': metrics, 'MEDIAN': medians, 'CI_LOW': ci_low, 'CI_HIGH': ci_high})
    return metrics

In [ ]:
n = 1000

In [ ]:
with open('nb_predictions.pkl', 'rb') as f:
    predictions = pickle.load(f)
    

nb_predictions_train = pd.DataFrame({'y_true': train_y, 
                                     'y_pred': predictions['train']['y_pred'], 
                                     'y_prob': predictions['train']['y_prob']})
nb_train_bootstrap_results = pd.DataFrame()
for i in range(n):
    nb_train_bootstrap_results = pd.concat([nb_train_bootstrap_results, 
                                            bootstrap_performance_metrics(nb_predictions_train)])
nb_train_metrics = summarize_bootstrap_results(nb_train_bootstrap_results)


nb_predictions_test = pd.DataFrame({'y_true': test_y, 
                                    'y_pred': predictions['test']['y_pred'], 
                                    'y_prob': predictions['test']['y_prob']})
nb_test_bootstrap_results = pd.DataFrame()
for i in range(n):
    nb_test_bootstrap_results = pd.concat([nb_test_bootstrap_results, 
                                           bootstrap_performance_metrics(nb_predictions_test)])
nb_test_metrics = summarize_bootstrap_results(nb_test_bootstrap_results)

In [ ]:
with open('lr_predictions.pkl', 'rb') as f:
    predictions = pickle.load(f)
    

lr_predictions_train = pd.DataFrame({'y_true': train_y, 
                                     'y_pred': predictions['train']['y_pred'], 
                                     'y_prob': predictions['train']['y_prob']})
lr_train_bootstrap_results = pd.DataFrame()
for i in range(n):
    lr_train_bootstrap_results = pd.concat([lr_train_bootstrap_results, 
                                            bootstrap_performance_metrics(lr_predictions_train)])
lr_train_metrics = summarize_bootstrap_results(lr_train_bootstrap_results)


lr_predictions_test = pd.DataFrame({'y_true': test_y, 
                                    'y_pred': predictions['test']['y_pred'], 
                                    'y_prob': predictions['test']['y_prob']})
lr_test_bootstrap_results = pd.DataFrame()
for i in range(n):
    lr_test_bootstrap_results = pd.concat([lr_test_bootstrap_results, 
                                           bootstrap_performance_metrics(lr_predictions_test)])
lr_test_metrics = summarize_bootstrap_results(lr_test_bootstrap_results)

In [ ]:
with open('rf_predictions.pkl', 'rb') as f:
    predictions = pickle.load(f)
    

rf_predictions_train = pd.DataFrame({'y_true': train_y, 
                                     'y_pred': predictions['train']['y_pred'], 
                                     'y_prob': predictions['train']['y_prob']})
rf_train_bootstrap_results = pd.DataFrame()
for i in range(n):
    rf_train_bootstrap_results = pd.concat([rf_train_bootstrap_results, 
                                            bootstrap_performance_metrics(rf_predictions_train)])
rf_train_metrics = summarize_bootstrap_results(rf_train_bootstrap_results)


rf_predictions_test = pd.DataFrame({'y_true': test_y, 
                                    'y_pred': predictions['test']['y_pred'], 
                                    'y_prob': predictions['test']['y_prob']})
rf_test_bootstrap_results = pd.DataFrame()
for i in range(n):
    rf_test_bootstrap_results = pd.concat([rf_test_bootstrap_results, 
                                           bootstrap_performance_metrics(rf_predictions_test)])
rf_test_metrics = summarize_bootstrap_results(rf_test_bootstrap_results)

In [ ]:
with open('nn_predictions.pkl', 'rb') as f:
    predictions = pickle.load(f)
    

nn_predictions_train = pd.DataFrame({'y_true': predictions['train']['y_true'], 
                                     'y_pred': predictions['train']['y_pred'], 
                                     'y_prob': predictions['train']['y_prob']})
nn_train_bootstrap_results = pd.DataFrame()
for i in range(n):
    nn_train_bootstrap_results = pd.concat([nn_train_bootstrap_results, 
                                            bootstrap_performance_metrics(nn_predictions_train)])
nn_train_metrics = summarize_bootstrap_results(nn_train_bootstrap_results)


nn_predictions_test = pd.DataFrame({'y_true': predictions['test']['y_true'], 
                                    'y_pred': predictions['test']['y_pred'], 
                                    'y_prob': predictions['test']['y_prob']})
nn_test_bootstrap_results = pd.DataFrame()
for i in range(n):
    nn_test_bootstrap_results = pd.concat([nn_test_bootstrap_results, 
                                           bootstrap_performance_metrics(nn_predictions_test)])
nn_test_metrics = summarize_bootstrap_results(nn_test_bootstrap_results)

In [ ]:
nb_train_metrics.to_csv('nb_train_metrics.csv', index=False)
nb_test_metrics.to_csv('nb_test_metrics.csv', index=False)

lr_train_metrics.to_csv('lr_train_metrics.csv', index=False)
lr_test_metrics.to_csv('lr_test_metrics.csv', index=False)

rf_train_metrics.to_csv('rf_train_metrics.csv', index=False)
rf_test_metrics.to_csv('rf_test_metrics.csv', index=False)

nn_train_metrics.to_csv('nn_train_metrics.csv', index=False)
nn_test_metrics.to_csv('nn_test_metrics.csv', index=False)